In [72]:
import ast
import os

import pandas as pd

our_dataset_path = '.'

posts_path = os.path.join(our_dataset_path, 'Data\\posts.csv')
fact_checks_path = os.path.join(our_dataset_path, 'Data\\fact_checks.csv')

for path in [posts_path, fact_checks_path]:
    print(f"Checking path: {path}")
    assert os.path.isfile(path), f"File not found: {path}"


Checking path: .\Data\posts.csv
Checking path: .\Data\fact_checks.csv


In [73]:
# #This is purely to sort the CVs numbers and not otherwise relevant to the main code - please disable for the code to work as intended
# import numpy as np

# df = pd.read_csv('pairs.csv', header=None)
# df.columns = ['post_id', 'fact_check_id']
# df.sort_values('post_id', ascending=False, inplace=True, key=lambda s: s.groupby(s).transform("size"))
# print (df.sort_values)

In [74]:
# parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s

# df_fact_checks = pd.read_csv(fact_checks_path).fillna('').set_index('fact_check_id')
# for col in ['claim', 'instances', 'title']:
#     df_fact_checks[col] = df_fact_checks[col].apply(parse_col)


# df_posts = pd.read_csv(posts_path).fillna('').set_index('post_id')
# for col in ['instances', 'ocr', 'verdicts', 'text']:
#     df_posts[col] = df_posts[col].apply(parse_col)

In [75]:
# All possible verdicts 
#  import csv

# # Der Pfad zur CSV-Datei
# csv_datei = posts_path

# # Öffnen der CSV-Datei und auslesen der 'verdicts' Spalte
# with open(csv_datei, newline='', encoding='utf-8') as csvfile:
#     # CSV-Reader erstellen
#     reader = csv.DictReader(csvfile)
    
#     # Set für einzigartige verdicts-Werte
#     verdicts_set = set()

#     # Iteriere durch jede Zeile in der CSV-Datei
#     for row in reader:
#         # Füge den Wert der 'verdicts'-Spalte zum Set hinzu
#         verdicts_set.add(row['verdicts'])

#     # Ausgabe der einzigartigen 'verdicts'-Werte
#     print("Mögliche 'verdicts'-Werte:")
#     for verdict in sorted(verdicts_set):
#         print(verdict)


In [76]:
## Print the output

# import pandas as pd

# # Load the CSV file
# df = pd.read_csv("merged_output.csv")

# # Set Pandas options to display full content without truncation
# pd.set_option("display.max_colwidth", None)  # No truncation for column width
# pd.set_option("display.max_rows", None)  # Show all rows (use with caution for large datasets)
# pd.set_option("display.max_columns", None)  # Show all columns
# pd.set_option("display.width", None)  # Automatically adjust width

# # Print the first row of every column
# for p in df.iloc[0].values:
#     print(p)  # Access the first row (index 0)


In [39]:
import ast
import pandas as pd

# Load CSV data

file_path_facts = "Data\\fact_checks.csv"
data_facts = pd.read_csv(file_path_facts)
file_path_posts = "Data\\posts.csv"
data_posts = pd.read_csv(file_path_posts,sep=",")

# Fill missing data
ocr_texts = data_posts['ocr'].fillna("")
titles = data_facts['claim'].fillna("")

ocr_texts = ocr_texts.apply(lambda x: ast.literal_eval(x))
titles = titles.apply(lambda x: ast.literal_eval(x))

# Create new lists for posts and facts
posts_list = []
for sublist in ocr_texts:
    for item in sublist:
        posts_list.append(item[0])

facts_list = []
for item in titles:
    facts_list.append(item[0])

post_ids = data_posts['post_id'].tolist()  # Extract post IDs
fact_check_ids = data_facts['fact_check_id'].tolist()  # Extract fact check IDs

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
ocr_vectors = vectorizer.fit_transform(facts_list)  # TF-IDF vectors for OCR texts
title_vectors = vectorizer.transform(posts_list)       # TF-IDF vectors for Titles

# Find Top 10 Most Similar Titles for Each OCR Text
top_k = 10
results_list = []

for i, ocr_text in enumerate(ocr_texts):
    # Compute cosine similarity between this OCR text and all titles
    similarities = cosine_similarity(ocr_vectors[i], title_vectors).flatten()
    
    # Get indices of top-k similar titles
    top_k_indices = similarities.argsort()[-top_k:][::-1]
    
    # Store the top-k titles and their similarity scores
    for idx in top_k_indices:
        results_list.append({
            "post_id": post_ids[i],  # Match post ID
            "ocr_text": ocr_text,
            "fact_check_id": fact_check_ids[idx],  # Match fact check ID
            "title": titles[idx],
            "similarity": similarities[idx]
        })

# Convert results to a DataFrame and save to CSV
results_df = pd.DataFrame(results_list)
results_df.to_csv("ocr_title_similarity.csv", index=False)
print("Results saved to ocr_title_similarity.csv")


Results saved to ocr_title_similarity.csv


In [ ]:
import json
import csv

# File paths
json_file = "Data\\monolingual_predictions.json"
csv_file = "ocr_title_similarity.csv"

# Load the JSON data
with open(json_file, "r", encoding="utf-8") as jf:
    monolingual_data = json.load(jf)

# Load the CSV data
csv_data = []
with open(csv_file, "r", encoding="utf-8") as cf:
    reader = csv.DictReader(cf)
    for row in reader:
        csv_data.append(row)

# Populate the JSON data with fact_check_id lists (convert to integers)
for post_id in monolingual_data.keys():
    fact_check_ids = [
        int(row["fact_check_id"]) for row in csv_data if row["post_id"] == post_id
    ]
    monolingual_data[post_id] = fact_check_ids

# Write the updated JSON data to a new file in one line
with open(json_file, "w", encoding="utf-8") as output_file:
    json.dump(monolingual_data, output_file, ensure_ascii=False, separators=(", ", ": "))

print(f"Updated JSON written to {json_file}")


Updated JSON written to Data\monolingual_predictions.json


In [48]:
import zipfile

# File paths to include in the zip
files_to_zip = ["monolingual_predictions.json"]  # Add more files as needed
output_zipfile = "predictions.zip"

# Create a zip file and add the specified files
with zipfile.ZipFile(output_zipfile, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file in files_to_zip:
        zipf.write(file, arcname=file)  # arcname keeps the same file name in the zip

print(f"Created {output_zipfile} containing: {', '.join(files_to_zip)}")


Created predictions.zip containing: monolingual_predictions.json
